# ICT3104 Project Team 12

## Setup
These libraries are required to run the tool.

In [1]:
#pip installs
pip install ruamel.yaml

# Import libraries
import os
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual
from IPython.display import display, clear_output, Video
import yaml
import ruamel.yaml

## Data Exploration
This section lets you select and view the videos from the data/input_files/training_videos folder, allowing you to verify if you have chosen the correct video.
To view the videos, select an option from the dropdown video and wait for the video to be displayed.

In [ ]:
# Define folder path of training videos
folder_path = "./data/input_files/training_videos/"

# Display video based on file chosen from the folder
@interact
def show_video(file=os.listdir(folder_path)):
    if(".mp4" not in file):
        print("Please choose a file with .mp4 format.")
        return
    video_path = f"{folder_path}{file}"
    video_display = Video(video_path, width=400, height=400, embed=True)
    display(video_display)

interactive(children=(Dropdown(description='file', options=('6R7QW.mp4', 'CCCUJ.mp4', 'HB9LP.mp4', 'JIR5V.mp4'…

## Inference

In [2]:
# Create a text input widget for pretrained_model_path
pretrained_model_path_input = widgets.Text(
    value="./checkpoints/stable-diffusion-v1-4",
    description="Pretrained Model Path:",
)

# Declare the validation_data dictionary
validation_data = {
    'prompts': [
        "Iron man on the beach",
        "Stormtrooper on the sea",
        "Astronaut on the beach"
    ],
    'video_length': 32,
    'width': 512,
    'height': 512,
    'num_inference_steps': 50,
    'guidance_scale': 12.5,
    'use_inv_latent': False,
    'num_inv_steps': 50,
    'dataset_set': 'val'
}

# Create a text input widget for prompts
prompts_input = widgets.Textarea(
    value = "\n".join(str(i) for i in (validation_data['prompts'])),
    description = "Prompts:",
    rows = 4
)

update_config_button = widgets.Button(description="Update Config")
yaml_file_path = "./ict3104-team12-2023/configs/pose_sample.yaml"
output_widget = widgets.Output()

def update_config(button_click):
    updated_path = pretrained_model_path_input.value
    updated_prompts = prompts_input.value.split("\n")

    # Read the existing YAML content from the file
    with open(yaml_file_path, 'r') as yaml_file:
        config_data = yaml.safe_load(yaml_file)

    config_data['pretrained_model_path'] = updated_path
    config_data['validation_data']['prompts'] = updated_prompts

    # Write the updated content back to the file
    with open(yaml_file_path, 'w') as yaml_file:
        yaml.dump(config_data, yaml_file, sort_keys=False)

    # Display the updated YAML content in the output widget
    with output_widget:
        clear_output()
        print(yaml.dump(config_data, sort_keys=False))

update_config_button.on_click(update_config)

# Display input fields
display(pretrained_model_path_input)
display(prompts_input)
# Preview
display(update_config_button, output_widget)

Text(value='./checkpoints/stable-diffusion-v1-4', description='Pretrained Model Path:')

Textarea(value='Iron man on the beach\nStormtrooper on the sea\nAstronaut on the beach', description='Prompts:…

Button(description='Update Config', style=ButtonStyle())

Output()

In [ ]:
video_path = "./ict3104-team12-2023/data/input_files/pose_examples/"

video_files = [f for f in os.listdir(video_path) if f.endswith(".mov")]

video_dropdown = widgets.Dropdown(
    options=video_files,
    description='Select a video:',
)

# Variable to store the selected video
selected_video = None

# Display the selected video and store the selection in the variable
def display_video(selected):
    global selected_video
    selected_video = selected
    video_file_path = os.path.join(video_path, selected)
    display(Video(video_file_path, width=400, height=300))

    # Update the print statement
    print("Selected video:", selected_video)

interactive(display_video, selected=video_dropdown)

## Training

In [ ]:
# Create a data dictionary for the content in the pose_train.yaml
data = {
    "pretrained_model_path": "./checkpoints/stable-diffusion-v1-4",
    "output_dir": "output",
    "train_data": {
        "video_path": "no path",
        "prompt": "None",
        "n_sample_frames": 12,
        "width": 512,
        "height": 512,
        "sample_start_idx": 0,
        "sample_frame_rate": 4,
        "dataset_set": "train",
    },
    "validation_data": {
        "prompts": [
            "A Iron man on the beach",
            "A Spider man on the snow",
            "A Superman on the street",
            "A boy on the forest",
        ],
        "video_length": 24,
        "width": 512,
        "height": 512,
        "num_inference_steps": 50,
        "guidance_scale": 12.5,
        "use_inv_latent": False,
        "num_inv_steps": 50,
        "dataset_set": "val",
    },
    "learning_rate": 3e-5,
    "train_batch_size": None,
    "num_train_epoch": None,
    "max_train_steps": 5000,
    "checkpointing_steps": 1000,
    "validation_steps": 100,
    "trainable_modules": [
        "attn1.to_q",
        "attn2.to_q",
        "attn_temp",
        "conv_temporal",
    ],
    "skeleton_path": "./pose_example/vis_kun_pose2.mov",
    "seed": 33,
    "mixed_precision": "no",
    "use_8bit_adam": False,
    "gradient_checkpointing": False,
    "enable_xformers_memory_efficient_attention": True,
}

# Define the generate pose_train.yaml function
def generate_yaml():
    
    #Convert the value in the text widget to integer (since by default it is treated as string)
    train_batch_size = int(batch_size_widget.value)
    num_train_epoch = int(epoch_widget.value)

    # Update the placeholders the respective values
    data["train_batch_size"] = train_batch_size
    data["num_train_epoch"] = num_train_epoch

    # Write the data to the pose_train.yaml file
    with open('./configs/pose_train.yaml', 'w') as file:
        yaml = ruamel.yaml.YAML()
        yaml.dump(data, file)

# Creation of the model name and batch size text and create model button widgets
name_widget = widgets.Text(description='Name:')
batch_size_widget = widgets.Text(description='Batch Size:')
epoch_widget = widgets.Text(description='Epochs:')
create_button = widgets.Button(description='Create Model')
create_button.on_click(generate_yaml)

# Display widgets
display(name_widget)
display(batch_size_widget)
display(epoch_widget)
display(create_button)

## Testing